## Imports

In [1]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import io
import boto3
from matplotlib import pyplot as plt

## Define Global Variables

In [2]:
client = boto3.client('s3')
bucket='obstacles-classification'
datasets = []

for prefix in client.list_objects(Bucket=bucket, Delimiter='/')['CommonPrefixes']:
    datasets.append(prefix['Prefix'][:-1])

## Define Widgets

In [3]:
dataset_dd = widgets.Dropdown(options=['<None>'] + datasets,
                              description='Dataset:',
                              value='<None>',
                              disabled=False,)
tog = widgets.ToggleButtons(options=['< prev', 'Obstacle', 'No Obstacle', 'Static', 'Removed', 'next >'])
im = widgets.Image()
upload_button = widgets.Button(description='Upload Annotations')
upload_button.style.button_color = 'lightgreen'
output = widgets.Output()

## Widget Callback Functions

In [4]:
# Variables used by Callback FUnctions
selected_dataset = None
image_names = []
cur_image = [0]
annotations = {}
tog_values = {
    'classidied_obstacle': 'Obstacle',
    'classified_no_obstacle': 'No Obstacle'
}

# Direct callback functions
def select_dataset(change):
    get_dataset_image_names(change['new'])

def tog_fn(change):
    handle_tog_fn(change['old'], change['new'])

def upload_annotations(change):
    changed = {key: value for (key, value) in annotations.items() if value['from'] != value['to']}
    with output:
        print(f'changed: {changed}')
        
# Utility functions used by callback functions
def get_dataset_image_names(dataset):
    for image_name in client.list_objects(Bucket=bucket, Prefix=dataset)['Contents']:
        if image_name['Key'].split('.')[-1] == 'jpg':
            image_names.append(image_name['Key'])
    display_image(image_names[cur_image[0]])
    annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
    annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
        
def display_image(imname):
    client = boto3.client("s3")
    bucket = 'obstacles-classification'
    key = imname
    outfile = io.BytesIO()
    client.download_fileobj(bucket, key, outfile)
    outfile.seek(0)
    bytes_data = outfile.read()
    im.value = bytes_data
    
def handle_tog_fn(old, new):
    tog_button_functions[new](old, new)
    
def prev_image(old, new):
    cur_image[0] = max(cur_image[0]-1, 0)
    display_image(image_names[cur_image[0]])
    annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
    annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
    
def annotate(old, new):
    annotations[cur_image[0]]['to'] = new
    
def next_image(old, new):
    cur_image[0] = min(cur_image[0]+1, len(image_names))
    display_image(image_names[cur_image[0]])
    annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
    annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
    
# Variable used by Toggle buttons handler
tog_button_functions = {
    '< prev': prev_image,
    'Obstacle': annotate,
    'No Obstacle': annotate,
    'Static': annotate,
    'Removed': annotate,
    'next >': next_image
}

## Define & Display Widgets

In [5]:
dataset_dd.observe(select_dataset, names='value')
tog.observe(tog_fn, names='value')
upload_button.on_click(upload_annotations)

display(dataset_dd, tog, im, upload_button, output)

Dropdown(description='Dataset:', options=('<None>', '', '3_class', 'debug_images', 'deeper_nn_experiment', 'ge…

ToggleButtons(options=('< prev', 'Obstacle', 'No Obstacle', 'Static', 'Removed', 'next >'), value='< prev')

Image(value=b'')

Button(description='Upload Annotations', style=ButtonStyle(button_color='lightgreen'))

Output()

KeyError: 'no_obstacle'

KeyError: 'no_obstacle'

KeyError: 'no_obstacle'